# 04_enrichment - ZE-zones en Cargo Bike Opportunities

Deze notebook analyseert:
1. Zero-Emissie zones en policy impact op cargo bike vraag
2. Urban Arrow opportuniteiten gebaseerd op ZE-beleid
3. Cargo bike markt prioritering en potentie scoring
4. Geographic clustering van ZE-policy impact

**Input**: Coverage data, white spots, ZE-zones policy data
**Output**: ZE-enriched opportunity scoring, cargo bike market analysis

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Directories
DATA_DIR = Path('../data')
OUTPUTS_DIR = Path('../outputs')
OUTPUTS_DIR.mkdir(exist_ok=True)
(OUTPUTS_DIR / 'tables').mkdir(exist_ok=True)
(OUTPUTS_DIR / 'plots').mkdir(exist_ok=True)

print("✅ Setup complete - ZE-zones & Cargo Bike Analysis")
print(f"Working directory: {Path.cwd()}")

✅ Setup complete - ZE-zones & Cargo Bike Analysis
Working directory: /Users/DINGZEEFS/Case_Gazelle_Pon/notebooks


## Data Loading & Preparation

Loading coverage data, white spots, en ZE-zones policy information

In [3]:
# Load coverage analysis results
try:
    white_spots = pd.read_csv(OUTPUTS_DIR / 'tables/white_spots_with_policy.csv')
except FileNotFoundError:
    # Fallback to basic white spots if policy version not available
    white_spots = pd.read_csv(OUTPUTS_DIR / 'tables/white_spots.csv')

gemeente_kpis = pd.read_csv(OUTPUTS_DIR / 'tables/gemeente_kpis.csv')
coverage_overall = pd.read_csv(OUTPUTS_DIR / 'tables/coverage_overall.csv')

# Load dealer data for Urban Arrow analysis
dealers = pd.read_parquet(DATA_DIR / 'processed/dealers.parquet')
urban_arrow_dealers = dealers[dealers['brand'] == 'Urban Arrow'].copy()

print(f"📊 Loaded data:")
print(f"   White spots: {len(white_spots)} locations")
print(f"   Gemeente KPIs: {len(gemeente_kpis)} municipalities")
print(f"   Urban Arrow dealers: {len(urban_arrow_dealers)} dealers")
print(f"   Coverage data: {len(coverage_overall)} distance bins")

# Display white spots with highest total scores
print("\n🎯 Top 10 white spots by total score:")
score_col = 'policy_score' if 'policy_score' in white_spots.columns else 'total_score'
if score_col in white_spots.columns:
    top_spots = white_spots.nlargest(10, score_col)[['pc4', 'plaats', 'gemeente', score_col, 'population']]
    print(top_spots.to_string(index=False))
else:
    print("   Score columns not found - will create new scoring")

📊 Loaded data:
   White spots: 66 locations
   Gemeente KPIs: 817 municipalities
   Urban Arrow dealers: 0 dealers
   Coverage data: 5 distance bins

🎯 Top 10 white spots by total score:
   Score columns not found - will create new scoring


## Zero-Emissie Zones Policy Analysis

Analyseer impact van ZE-beleid op cargo bike demand en Urban Arrow opportunities

In [4]:
# Define Zero-Emission zones and policy impact scores
# Based on Dutch cities implementing ZE/LEZ zones 2025-2030
ze_policy_data = {
    'Amsterdam': {'implementation_year': 2025, 'scope': 'city_center', 'policy_strength': 10},
    'Rotterdam': {'implementation_year': 2025, 'scope': 'city_center', 'policy_strength': 9},
    'Den Haag': {'implementation_year': 2025, 'scope': 'city_center', 'policy_strength': 9},
    'Utrecht': {'implementation_year': 2025, 'scope': 'city_center', 'policy_strength': 10},
    'Eindhoven': {'implementation_year': 2026, 'scope': 'city_center', 'policy_strength': 8},
    'Tilburg': {'implementation_year': 2026, 'scope': 'city_center', 'policy_strength': 7},
    'Groningen': {'implementation_year': 2026, 'scope': 'city_center', 'policy_strength': 8},
    'Almere': {'implementation_year': 2027, 'scope': 'city_center', 'policy_strength': 6},
    'Nijmegen': {'implementation_year': 2027, 'scope': 'city_center', 'policy_strength': 7},
    'Enschede': {'implementation_year': 2027, 'scope': 'city_center', 'policy_strength': 6},
    'Haarlem': {'implementation_year': 2028, 'scope': 'city_center', 'policy_strength': 6},
    'Arnhem': {'implementation_year': 2028, 'scope': 'city_center', 'policy_strength': 5},
    'Zaanstad': {'implementation_year': 2028, 'scope': 'partial', 'policy_strength': 4},
    'Maastricht': {'implementation_year': 2029, 'scope': 'city_center', 'policy_strength': 5},
    'Dordrecht': {'implementation_year': 2029, 'scope': 'partial', 'policy_strength': 4},
    'Leiden': {'implementation_year': 2029, 'scope': 'city_center', 'policy_strength': 5},
    'Amersfoort': {'implementation_year': 2030, 'scope': 'partial', 'policy_strength': 4},
    'Zwolle': {'implementation_year': 2030, 'scope': 'partial', 'policy_strength': 3}
}

# Convert to DataFrame
ze_policies_df = pd.DataFrame.from_dict(ze_policy_data, orient='index').reset_index()
ze_policies_df.columns = ['gemeente', 'implementation_year', 'scope', 'policy_strength']

# Calculate time-adjusted policy score (earlier = higher urgency)
current_year = 2025
ze_policies_df['years_until_implementation'] = ze_policies_df['implementation_year'] - current_year
ze_policies_df['urgency_multiplier'] = np.maximum(1, 5 - ze_policies_df['years_until_implementation'])
ze_policies_df['adjusted_policy_score'] = ze_policies_df['policy_strength'] * ze_policies_df['urgency_multiplier']

# Scope multipliers
scope_multipliers = {'city_center': 1.0, 'partial': 0.7}
ze_policies_df['scope_multiplier'] = ze_policies_df['scope'].map(scope_multipliers)
ze_policies_df['final_policy_score'] = ze_policies_df['adjusted_policy_score'] * ze_policies_df['scope_multiplier']

print("🏛️ Zero-Emission Policy Scoring:")
display_df = ze_policies_df[['gemeente', 'implementation_year', 'policy_strength', 'urgency_multiplier', 'final_policy_score']].round(2)
print(display_df.to_string(index=False))

# Merge with gemeente KPIs
gemeente_kpis_enriched = gemeente_kpis.merge(ze_policies_df[['gemeente', 'final_policy_score']], 
                                            on='gemeente', how='left')
gemeente_kpis_enriched['final_policy_score'] = gemeente_kpis_enriched['final_policy_score'].fillna(0)

print(f"\n✅ Enriched {len(gemeente_kpis_enriched)} gemeenten with ZE-policy scores")
print(f"   {len(gemeente_kpis_enriched[gemeente_kpis_enriched['final_policy_score'] > 0])} gemeenten have ZE-policies")

🏛️ Zero-Emission Policy Scoring:
  gemeente  implementation_year  policy_strength  urgency_multiplier  final_policy_score
 Amsterdam                 2025               10                   5                50.0
 Rotterdam                 2025                9                   5                45.0
  Den Haag                 2025                9                   5                45.0
   Utrecht                 2025               10                   5                50.0
 Eindhoven                 2026                8                   4                32.0
   Tilburg                 2026                7                   4                28.0
 Groningen                 2026                8                   4                32.0
    Almere                 2027                6                   3                18.0
  Nijmegen                 2027                7                   3                21.0
  Enschede                 2027                6                   3         

In [ ]:
# Cargo bike opportunity scoring
cargo_opportunities = gemeente_kpis_enriched.copy()

# Rename columns to match expected names
if 'pop_total' in cargo_opportunities.columns:
    cargo_opportunities['population'] = cargo_opportunities['pop_total']

# First, create a mapping from PC4 to gemeente using available data
# Load demografie data which should have PC4 to gemeente mapping
demografie = pd.read_parquet(DATA_DIR / 'processed/demografie.parquet')
pc4_gemeente_mapping = demografie[['pc4', 'gemeente']].drop_duplicates().copy()

# Add gemeente to dealers based on PC4
dealers_with_gemeente = dealers.merge(pc4_gemeente_mapping, on='pc4', how='left')

# Add Urban Arrow dealer counts per gemeente if not present
if 'urban_arrow_dealers' not in cargo_opportunities.columns:
    ua_counts = dealers_with_gemeente[dealers_with_gemeente['brand'] == 'Urban Arrow'].groupby('gemeente').size().reset_index(name='urban_arrow_dealers')
    cargo_opportunities = cargo_opportunities.merge(ua_counts, on='gemeente', how='left')
    cargo_opportunities['urban_arrow_dealers'] = cargo_opportunities['urban_arrow_dealers'].fillna(0)

# Add urbanization column if not present (use density as proxy)
if 'urbanization' not in cargo_opportunities.columns:
    # Create urbanization categories based on density
    cargo_opportunities['urbanization'] = pd.cut(
        cargo_opportunities['density_norm'],
        bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
        labels=['Niet stedelijk', 'Weinig stedelijk', 'Matig stedelijk', 'Sterk stedelijk', 'Zeer stedelijk']
    ).astype(str)

# Add competition_index if not present (use inverse of pon_share as proxy)
if 'competition_index' not in cargo_opportunities.columns:
    cargo_opportunities['competition_index'] = 1 - cargo_opportunities['pon_share']

# 1. ZE-Policy Score (30% weight)
max_policy_score = cargo_opportunities['final_policy_score'].max()
cargo_opportunities['ze_score_normalized'] = (
    cargo_opportunities['final_policy_score'] / max_policy_score if max_policy_score > 0 else 0
)
cargo_opportunities['ze_score_normalized'] = cargo_opportunities['ze_score_normalized'].fillna(0)

# 2. Demographic Score - use population density and urbanization as proxies (25% weight)
cargo_opportunities['pop_density'] = cargo_opportunities['population'] / 1000  # Simplified density metric
max_density = cargo_opportunities['pop_density'].max()
urbanization_scores = {'Zeer stedelijk': 1.0, 'Sterk stedelijk': 0.8, 'Matig stedelijk': 0.6, 'Weinig stedelijk': 0.4, 'Niet stedelijk': 0.2}

cargo_opportunities['demographic_score_raw'] = (
    0.6 * (cargo_opportunities['pop_density'] / max_density) +
    0.4 * cargo_opportunities['urbanization'].map(urbanization_scores).fillna(0.4)
)

# 3. Market Gap Score - low Urban Arrow presence = higher opportunity (20% weight)
cargo_opportunities['ua_density_per_100k'] = (cargo_opportunities['urban_arrow_dealers'] / cargo_opportunities['population']) * 100000
max_ua_density = cargo_opportunities['ua_density_per_100k'].quantile(0.9)  # Use 90th percentile as max
if max_ua_density > 0:
    cargo_opportunities['market_gap_score'] = 1 - (cargo_opportunities['ua_density_per_100k'] / max_ua_density).clip(0, 1)
else:
    cargo_opportunities['market_gap_score'] = 1.0  # No UA presence = maximum gap

# 4. Competition Score - lower competition = higher opportunity (15% weight)
max_competition = cargo_opportunities['competition_index'].max()
cargo_opportunities['competition_score'] = 1 - (cargo_opportunities['competition_index'] / max_competition).fillna(0.5)

# 5. Population Scale Score (10% weight)
max_population = cargo_opportunities['population'].max()
cargo_opportunities['scale_score'] = cargo_opportunities['population'] / max_population

# Weighted final score
weights = {
    'ze_score_normalized': 0.30,
    'demographic_score_raw': 0.25,
    'market_gap_score': 0.20,
    'competition_score': 0.15,
    'scale_score': 0.10
}

cargo_opportunities['cargo_bike_opportunity_score'] = 0
for col, weight in weights.items():
    cargo_opportunities['cargo_bike_opportunity_score'] += cargo_opportunities[col] * weight

# Scale to 0-100
cargo_opportunities['cargo_bike_opportunity_score'] *= 100

# Categorize opportunities
cargo_opportunities['opportunity_category'] = pd.cut(
    cargo_opportunities['cargo_bike_opportunity_score'],
    bins=[0, 25, 50, 75, 100],
    labels=['Low', 'Medium', 'High', 'Prime']
)

print("🚲 Cargo Bike Opportunity Scoring Complete:")
print("\n📊 Opportunity Distribution:")
print(cargo_opportunities['opportunity_category'].value_counts().to_string())

print("\n🎯 Top 15 Cargo Bike Opportunities:")
top_cargo_opps = cargo_opportunities.nlargest(15, 'cargo_bike_opportunity_score')[
    ['gemeente', 'population', 'final_policy_score', 'pon_dealers', 'urban_arrow_dealers', 
     'cargo_bike_opportunity_score', 'opportunity_category']
]
print(top_cargo_opps.round(2).to_string(index=False))

SyntaxError: unexpected character after line continuation character (1746038257.py, line 91)

In [ ]:
# Export cargo bike opportunity analysis
export_cols = ['gemeente', 'population', 'urbanization', 'pon_dealers', 'urban_arrow_dealers',
               'final_policy_score', 'ze_score_normalized', 'demographic_score_raw', 
               'market_gap_score', 'competition_score', 'scale_score',
               'cargo_bike_opportunity_score', 'opportunity_category']

# Only include columns that exist
available_cols = [col for col in export_cols if col in cargo_opportunities.columns]
cargo_export = cargo_opportunities[available_cols].copy()

# Round scores for readability
score_cols = ['final_policy_score', 'ze_score_normalized', 'demographic_score_raw', 
              'market_gap_score', 'competition_score', 'scale_score', 'cargo_bike_opportunity_score']
for col in score_cols:
    if col in cargo_export.columns:
        cargo_export[col] = cargo_export[col].round(2)

cargo_export.to_csv(OUTPUTS_DIR / 'tables/cargo_bike_opportunities.csv', index=False)

# Export ZE-policy summary
ze_summary = ze_policies_df[[
    'gemeente', 'implementation_year', 'scope', 'policy_strength', 'final_policy_score'
]].copy()
ze_summary.to_csv(OUTPUTS_DIR / 'tables/ze_policy_impact.csv', index=False)

print(f"\n✅ Analysis complete - files exported:")
print(f"   📊 cargo_bike_opportunities.csv")
print(f"   🏛️ ze_policy_impact.csv")
print(f"\n🚲 ZE-zones & Cargo Bike Enrichment Analysis Complete!")